In [6]:
import numpy as np
from annoy import AnnoyIndex
import io

In [7]:
class PreTrainedEmbeddings(object):
    def __init__(self, word_to_index, word_vectors):
        """
        Аргументы:
            word_to_index (dict): отображение слов
            в целочисленные значения
            word_vectors (список массивов numpy)
        """
        self.word_to_index = word_to_index
        self.word_vectors = word_vectors
        self.index_to_words = {
            v: k 
            for k, v in self.word_to_index.items()
        }
        self.index = AnnoyIndex(len(word_vectors[0]), metric="euclidean")
        for _, i in self.word_to_index.items():
            self.index.add_item(i, self.word_vectors[i])
        self.index.build(50)
    
    @classmethod
    def from_embeddings_file(cls, embeddings_file):
        """ Создаем экземпляр на основе файла векторов,
            заранее полученных в результате обучения
        Формат файла векторов должен быть следующим:
            word0 x0_0 x0_1 x0_2 x0_3 ... x0_N
            word1 x1_0 x1_1 x1_2 x1_3 ... x1_N
            
        Аргументы:
            embedding_file (str): местоположение файла
        Возвращает:
            экземпляр PretrainedEmbeddings
        """
        word_to_index = dict()
        word_vectors = []
        with io.open(embeddings_file, encoding='utf-8') as fp:
            for line in fp.readlines():
                line = line.split(" ")
                word = line[0]
                vec = np.array([float(x) for x in line[1:]])
                
                word_to_index[word] = len(word_to_index)
                word_vectors.append(vec)
        return cls(word_to_index, word_vectors)

Будет решаться задача на анологию дается два слова, как то объединенные вместе, так же дается третье слово, Задача: найти четвертое слово, которое связано с третьим так же, как первое со вторым.

In [21]:
class PreTrainedEmbeddings(object):
    def __init__(self, word_to_index, word_vectors):
        """
        Аргументы:
            word_to_index (dict): отображение слов
            в целочисленные значения
            word_vectors (список массивов numpy)
        """
        self.word_to_index = word_to_index
        self.word_vectors = word_vectors
        self.index_to_word = {
            v: k 
            for k, v in self.word_to_index.items()
        }
        self.index = AnnoyIndex(len(word_vectors[0]), metric="euclidean")
        for _, i in self.word_to_index.items():
            self.index.add_item(i, self.word_vectors[i])
        self.index.build(50)
    
    @classmethod
    def from_embeddings_file(cls, embeddings_file):
        """ Создаем экземпляр на основе файла векторов,
            заранее полученных в результате обучения
        Формат файла векторов должен быть следующим:
            word0 x0_0 x0_1 x0_2 x0_3 ... x0_N
            word1 x1_0 x1_1 x1_2 x1_3 ... x1_N
            
        Аргументы:
            embedding_file (str): местоположение файла
        Возвращает:
            экземпляр PretrainedEmbeddings
        """
        word_to_index = dict()
        word_vectors = []
        with io.open(embeddings_file, encoding='utf-8') as fp:
            for line in fp.readlines():
                line = line.split(" ")
                word = line[0]
                vec = np.array([float(x) for x in line[1:]])
                
                word_to_index[word] = len(word_to_index)
                word_vectors.append(vec)
        return cls(word_to_index, word_vectors)
    
    def get_embeddings(self, word):
        return self.word_vectors[self.word_to_index[word]]
    
    def get_closest_to_vector(self, vector, n=1):
        nn_indices = self.index.get_nns_by_vector(vector, n)
        return [self.index_to_word[neighbor] for neighbor in nn_indices]
    
    def compute_and_print_analogy(self, word_1, word_2, word_3):
        vec1 = self.get_embeddings(word_1)
        vec2 = self.get_embeddings(word_2)
        vec3 = self.get_embeddings(word_3)
        
        spartial_relations = vec2 - vec1
        vec4 = vec3 + spartial_relations
        
        closest_words = self.get_closest_to_vector(vec4, n=4)
        existing_words = set([word_1, word_2, word_3])
        closest_words = [word for word in closest_words if word not in existing_words]
        
        if len(closest_words) == 0:
            print("Could not find closest words")
            return 
        
        for word4 in closest_words:
            print(f"{word_1} : {word_2} :: {word_3} : {word4}")

In [22]:
embeddings = PreTrainedEmbeddings.from_embeddings_file("../data/embeddings/glove.6B.100d.txt")

In [24]:
embeddings.compute_and_print_analogy('man', 'he', 'woman')

man : he :: woman : she
man : he :: woman : her


In [31]:
embeddings.compute_and_print_analogy('fly', 'plane', 'sail')

fly : plane :: sail : ship
fly : plane :: sail : vessel


In [32]:
embeddings.compute_and_print_analogy('cat', 'kitten', 'dog')

cat : kitten :: dog : puppy
cat : kitten :: dog : pooch
cat : kitten :: dog : junkyard


In [36]:
embeddings.compute_and_print_analogy('green', 'color', 'dog')

green : color :: dog : viewer
green : color :: dog : creature
green : color :: dog : images


In [37]:
embeddings.compute_and_print_analogy('toe', 'foot', 'finger')

toe : foot :: finger : hand
toe : foot :: finger : fingers


In [38]:
embeddings.compute_and_print_analogy('talk', 'communicate', 'read')

talk : communicate :: read : typed
talk : communicate :: read : copy
talk : communicate :: read : anonymously


In [39]:
embeddings.compute_and_print_analogy('blue', 'democrat', 'red')

blue : democrat :: red : republican
blue : democrat :: red : congressman
blue : democrat :: red : senator


In [41]:
embeddings.compute_and_print_analogy('fast', 'fastest', 'young')

fast : fastest :: young : youngest
fast : fastest :: young : fellow
fast : fastest :: young : younger


In [43]:
embeddings.compute_and_print_analogy('fast', 'fastest', 'small')

fast : fastest :: small : third
fast : fastest :: small : fifth
fast : fastest :: small : fourth


In [44]:
embeddings.compute_and_print_analogy('man', 'king', 'woman')

man : king :: woman : queen
man : king :: woman : monarch
man : king :: woman : throne


In [45]:
embeddings.compute_and_print_analogy('man', 'doctor', 'woman')

man : doctor :: woman : nurse
man : doctor :: woman : physician
